# Init Colab

## The path needs to be adapted if there is need to re-run. The given implementation was run on google drive from google colab.

In [1]:
# TOUJOURS RUN
# Get Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.chdir('/content/drive/MyDrive/NLP_PROJECT/')

Mounted at /content/drive


In [2]:
! git clone https://github.com/openai/human-eval
! pip install -e human-eval
!pip install backoff

%cd human-eval
!pip install --upgrade pip


fatal: destination path 'human-eval' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/.shortcut-targets-by-id/1SM-QTkF2Le4AUTo1O-tGUCya71N_n7d7/NLP_PROJECT/human-eval
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=c116ff99724adcad6e3a4a59b2f3053957333df6aa25bc7e44025d54cd113406
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
  Running setup.py develop for human-eval
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/drive/.shortcut-targets-by-id/1SM-QTkF2Le4AUTo1O-tGUCya71N_n7d7/NLP_PROJECT/human-eval
Looking in indexes: http

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.4 MB/s eta 0:00:00


# Skip

In [ ]:
# Test
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen2-1B")
model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen2-1B", trust_remote_code=True, revision="main")


In [ ]:
# Test
inputs = tokenizer("# this function prints a random number between 1 and 5", return_tensors="pt")
sample = model.generate(**inputs, max_length=128)
print(tokenizer.decode(sample[0], truncate_before_pattern=[r"\n\n^#", "^'''", "\n\n\n"]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


# this function prints a random number between 1 and 5
        def random_number():
            return random.randint(1, 5)

        #


In [ ]:
# Test
def generate_one_completion2(prompt):
  inputs = tokenizer(prompt, return_tensors="pt")
  sample = model.generate(**inputs, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id)
  return tokenizer.decode(sample[0], truncate_before_pattern=[r"\n\n^#", "^'''", "\n\n\n"])

In [ ]:
# Test
def generate_one_completion(prompt):
  # Tokenize the prompt
  inputs = tokenizer(prompt, return_tensors="pt")

  print('-------------------------------')
  print("Prompt :\n",prompt)

  # Set the initial value of max_new_tokens
  max_new_tokens = 32
  print("trying with max_tokens ", max_new_tokens)
  # Generate completion with the current value of max_new_tokens
  sample = model.generate(**inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
  completion_tokens = sample[0, len(inputs[0]):]
  completion = tokenizer.decode(completion_tokens, truncate_before_pattern=[r"\n\n^#", "^'''", "\n\n\n"])


  print("Completion :\n", completion)
  print("len of completion :", len(completion_tokens))
  # Check if the completion was truncated
  while len(completion_tokens) == max_new_tokens:
    # Increase max_new_tokens
    max_new_tokens += 16
    print("trying with max_tokens ", max_new_tokens)

    # Generate completion with the updated value of max_new_tokens
    sample = model.generate(**inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    completion_tokens = sample[0, len(inputs[0]):]
    completion = tokenizer.decode(completion_tokens,truncate_before_pattern=[r"\n\n^#", "^'''", "\n\n\n"])
    print("Completion :\n", completion)
    print("len of completion :", len(completion_tokens))

  return completion


# HERE

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device="cuda" if torch.cuda.is_available() else "cpu"
print("device :",device)

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen2-1B")
model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen2-1B", trust_remote_code=True, revision="main").to(device)


device : cuda


In [5]:
def generate_one_completion(prompt,num_max_token, device):
  # Tokenize the prompt
  inputs = tokenizer(prompt, return_tensors="pt").to(device)

  print('-------------------------------')
  print("Prompt :\n",prompt)

  input_len=len(inputs[0])

  # Generate completion
  sample = model.generate(**inputs, do_sample=True, temperature=0.2, top_p=0.95,
                          max_length=input_len+num_max_token, pad_token_id=tokenizer.eos_token_id)
  sample = sample[0,input_len:]
  completion = tokenizer.decode(sample, truncate_before_pattern=[r"\n\n^#", "^'''", "\n\n\n"])


  print("Completion :\n", completion)

  return completion


In [6]:
from human_eval.data import write_jsonl, read_problems
data = read_problems()
pb_id = 4
pb_prompt = data['HumanEval/'+str(pb_id)]['prompt']

In [ ]:
# SKIP
sample_sol = generate_one_completion(pb_prompt, device)
print(sample_sol)

In [7]:
import json

def add_json_to_jsonl_file(file_path, new_json_object):
    # Step 1: Open the JSONL file in append mode
    with open(file_path, 'a') as file:
        # Step 2: Serialize the new JSON object into a string
        json_string = json.dumps(new_json_object)

        # Step 3: Write the serialized JSON object as a new line in the file
        file.write(json_string)
        file.write('\n')
        print("results stored in ",file_path)

In [8]:
problems = read_problems()
for task_id in problems:
  print(task_id)
  break

HumanEval/0


In [14]:
import os
current_dir = os.getcwd()
path_dir = os.path.join((os.path.dirname(current_dir)),"codeGen_output/")
file_path=os.path.join(path_dir,"codeGen_output.jsonl")

if os.path.exists(file_path):
    print("The file {} already exists.".format(file_path))
else :
  print("Results will be stored in {}".format(file_path))


problems = read_problems()

num_samples_per_task = 3
num_max_token=128
samples = []
humanEval_samples=[]
output={}
counter = 0
for task_id in problems:
  if counter <123:
    counter +=1
    continue
  for _ in range(num_samples_per_task):

    # Make prediction
    completion = generate_one_completion(problems[task_id]["prompt"],num_max_token,device)
    # File for codeT
    samples.append(completion)
    # File for HumanEval
    humanEval_samples.append(dict(task_id=task_id, completion=completion))
  res={"prompt":problems[task_id]["prompt"],"samples":samples}
  print("\n-----------------------\n")
  add_json_to_jsonl_file(file_path,res)
  counter +=1
  print('Problems Attempted: ' + str(counter))



write_jsonl(os.path.join(path_dir,"samples.jsonl"), humanEval_samples)

The file /content/drive/.shortcut-targets-by-id/1SM-QTkF2Le4AUTo1O-tGUCya71N_n7d7/NLP_PROJECT/codeGen_output/codeGen_output.jsonl already exists.
-------------------------------
Prompt :
 
def get_odd_collatz(n):
    """
    Given a positive integer n, return a sorted list that has the odd numbers in collatz sequence.

    The Collatz conjecture is a conjecture in mathematics that concerns a sequence defined
    as follows: start with any positive integer n. Then each term is obtained from the 
    previous term as follows: if the previous term is even, the next term is one half of 
    the previous term. If the previous term is odd, the next term is 3 times the previous
    term plus 1. The conjecture is that no matter what value of n, the sequence will always reach 1.

    Note: 
        1. Collatz(1) is [1].
        2. returned list sorted in increasing order.

    For example:
    get_odd_collatz(5) returns [1, 5] # The collatz sequence for 5 is [5, 16, 8, 4, 2, 1], so the odd numb

# STOP HERE


In [ ]:
import os
current_dir = os.getcwd()
path_dir = os.path.join((os.path.dirname(current_dir)),"codeGen_output/")
file_path=os.path.join(path_dir,"codeGen_output.json")

if os.path.exists(file_path):
    print("The file {} already exists.".format(file_path))
else :
  print("Results will be stored in {}".format(file_path))


problems = read_problems()

num_samples_per_task = 3
num_max_token=128
samples = []
humanEval_samples=[]
output={}
counter = 0
for task_id in problems:
  for _ in range(num_samples_per_task):

    # Make prediction
    completion = generate_one_completion(problems[task_id]["prompt"],num_max_token,device)
    # File for codeT
    samples.append(completion)
    # File for HumanEval
    humanEval_samples.append(dict(task_id=task_id, completion=completion))
  res={"prompt":problems[task_id]["prompt"],"samples":samples}
  print("\n-----------------------\n")
  add_json_to_jsonl_file(file_path,res)
  counter +=1
  print('Problems Attempted: ' + str(counter))



write_jsonl(os.path.join(path_dir,"samples.jsonl"), humanEval_samples)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
def strange_sort_list_2(lst):
    '''
    Given list of integers, return list in strange order.
    Strange sorting, is when you start with the minimum value,
    then maximum of the remaining integers, then minimum and so on.

    Examples:
    strange_sort_list([1, 2, 3, 4]) == [1, 4,
-------------------------------
Prompt :
 
def strange_sort_list(lst):
    '''
    Given list of integers, return list in strange order.
    Strange sorting, is when you start with the minimum value,
    then maximum of the remaining integers, then minimum and so on.

    Examples:
    strange_sort_list([1, 2, 3, 4]) == [1, 4, 2, 3]
    strange_sort_list([5, 5, 5, 5]) == [5, 5, 5, 5]
    strange_sort_list([]) == []
    '''

Completion :
     min_val = min(lst)
    max_val = max(lst)
    min_val_idx = lst.index(min_val)
    max_val_idx = lst.index(max_val)
    return lst[min_val_idx:max_val_idx+1]

def strange_sort_list_2(lst):

## old one

In [ ]:
import os
current_dir = os.getcwd()
path_dir = os.path.join("codeGen_output/",(os.path.dirname(current_dir)))
file_path=os.path.join(path_dir,"codeGen_output.json")

if os.path.exists(file_path):
    print("The file {} already exists.".format(file_path))
else :
  print("Results will be stored in {}".format(file_path))


problems = read_problems()

num_samples_per_task = 3
num_max_token=128
samples = []
humanEval_samples=[]
output={}
counter = 0
for task_id in problems:
  for _ in range(num_samples_per_task):

    # Make prediction
    completion = generate_one_completion(problems[task_id]["prompt"],num_max_token,device)
    # File for codeT
    samples.append(completion)
    # File for HumanEval
    humanEval_samples.append(dict(task_id=task_id, completion=completion))
  res={"prompt":problems[task_id]["prompt"],"samples":samples}
  print("\n-----------------------\n")
  add_json_to_jsonl_file(file_path,res)
  counter +=1
  print('Problems Attempted: ' + str(counter))



write_jsonl(os.path.join(path_dir,"samples.jsonl"), humanEval_samples)

Streaming output truncated to the last 5000 lines.
    >>> split('a', '')
    ['a']
    >>> split('a', 'b')
    ['a', 'b']
    """
    return [string]


def join(strings: List[str], separator: str) -> str:
    """ Join list of strings into a single string
    >>> join([], '')
    
-------------------------------
Prompt :
 from typing import List


def concatenate(strings: List[str]) -> str:
    """ Concatenate list of strings into a single string
    >>> concatenate([])
    ''
    >>> concatenate(['a', 'b', 'c'])
    'abc'
    """

Completion :
     return ''.join(strings)


def split(string: str, separator: str) -> List[str]:
    """ Split a string into a list of substrings
    >>> split('abc', 'a')
    ['abc', 'a']
    >>> split('abc', 'b')
    ['abc', 'b']
    >>> split('abc', 'c')
    ['abc', 'c']
    """
    return string.split(separator)


def join(strings: List[str]) -> str:
    """ Join a list of strings
-------------------------------
Prompt :
 from typing import List


def co

In [ ]:
problems = read_problems()

num_samples_per_task = 1
samples = []
counter = 0
for task_id in problems:
  for _ in range(num_samples_per_task):
    samples.append(dict(task_id=task_id, completion= generate_one_completion2(problems[task_id]["prompt"])))
  counter +=1
  print('Problems Attempted: ' + str(counter))



write_jsonl("samples.jsonl", samples)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Problems Attempted: 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Problems Attempted: 2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Problems Attempted: 23


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Problems Attempted: 24


KeyboardInterrupt: ignored

In [ ]:
! evaluate_functional_correctness samples.jsonl